In [23]:
import nest_asyncio

nest_asyncio.apply()

from IPython.display import display, HTML

In [24]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3:8b", request_timeout=60.0)

In [5]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(role="system", content="You are a helpful assistant."),
    ChatMessage(role="user", content="What are the most important landmarks in France?"),
]
response = llm.chat(messages)

In [8]:
display(HTML(f'<p style="font-size:20px">{response}</p>'))

In [9]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [10]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [11]:
!wget "https://www.dropbox.com/scl/fi/ywc29qvt66s8i97h1taci/lyft-10k-2020.pdf?rlkey=d7bru2jno7398imeirn09fey5&dl=0" -q -O ./lyft_10k_2020.pdf
!wget "https://www.dropbox.com/scl/fi/lpmmki7a9a14s1l5ef7ep/lyft-10k-2021.pdf?rlkey=ud5cwlfotrii6r5jjag1o3hvm&dl=0" -q -O ./lyft_10k_2021.pdf
!wget "https://www.dropbox.com/scl/fi/iffbbnbw9h7shqnnot5es/lyft-10k-2022.pdf?rlkey=grkdgxcrib60oegtp4jn8hpl8&dl=0" -q -O ./lyft_10k_2022.pdf

In [12]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

lyft_2020_docs = SimpleDirectoryReader(input_files=["./lyft_10k_2020.pdf"]).load_data()
lyft_2020_index = VectorStoreIndex.from_documents(lyft_2020_docs)
lyft_2020_engine = lyft_2020_index.as_query_engine()

lyft_2021_docs = SimpleDirectoryReader(input_files=["./lyft_10k_2021.pdf"]).load_data()
lyft_2021_index = VectorStoreIndex.from_documents(lyft_2021_docs)
lyft_2021_engine = lyft_2021_index.as_query_engine()

lyft_2022_docs = SimpleDirectoryReader(input_files=["./lyft_10k_2022.pdf"]).load_data()
lyft_2022_index = VectorStoreIndex.from_documents(lyft_2022_docs)
lyft_2022_engine = lyft_2022_index.as_query_engine()

response = lyft_2022_engine.query("What was Lyft's profit in 2022?")
print(response)

Based on the provided financial statements, Lyft reported a net loss of $1,584,511 for the year ended December 31, 2022. Therefore, there was no profit in 2022.


In [13]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_2020_engine,
        metadata=ToolMetadata(
            name="lyft_2020_10k_form",
            description="Annual report of Lyft's financial activities in 2020",
        ),
    ),
    QueryEngineTool(
        query_engine=lyft_2021_engine,
        metadata=ToolMetadata(
            name="lyft_2021_10k_form",
            description="Annual report of Lyft's financial activities in 2021",
        ),
    ),
    QueryEngineTool(
        query_engine=lyft_2022_engine,
        metadata=ToolMetadata(
            name="lyft_2022_10k_form",
            description="Annual report of Lyft's financial activities in 2022",
        ),
    ),
]

In [18]:
from llama_index.core.agent import ReActAgent

lyft_agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True, max_iterations=20)
response = lyft_agent.chat("What are the risk factors in 2022? Use existing tools.")
print(response)

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: lyft_2022_10k_form
Action Input: {'input': 'risk factors'}
Observation: Our business operations are subject to numerous risks, factors, and uncertainties that could cause our actual results to be harmed, including those outside of our control. These risks and uncertainties relate to general economic factors such as the impact of the COVID-19 pandemic and responsive measures, general macroeconomic conditions, natural disasters, economic downturns, public health crises or political crises.
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The risk factors in 2022 for Lyft mentioned in the annual report include those outside of their control such as the impact of the COVID-19 pandemic and responsive measures, general macroeconomic conditions, natural disasters, economic downturns, public health crises or political crises.
The 

In [20]:
from llama_index.core.agent import ReActAgent

lyft_agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True, max_iterations=20)
response = lyft_agent.chat("What is Lyft's profit in 2022 vs 2020? Generate only one step at a time. Use existing tools.")
print(response)

Thought: The current language of the user is: unknown. I need to use a tool to help me answer the question.
Action: lyft_2022_10k_form
Action Input: {'input': 'profit', 'num_beams': 5}
Observation: Revenue.
Thought: The user provided information is not the profit, but revenue. I need to use another tool to get the profit.
Action: lyft_2020_10k_form
Action Input: {'input': 'profit', 'num_beams': 5}
Observation: The company reported a net loss of $1,752,857 for the year ended December 31, 2020.
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Lyft's profit in 2020 is not available as it reported a net loss.
Lyft's profit in 2020 is not available as it reported a net loss.


In [25]:
from llama_index.core.agent import ReActAgent

lyft_agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True, max_iterations=20)
response = lyft_agent.chat("What did Lyft do in R&D in 2022 versus 2021? Generate only one step at a time. Use existing tools.")
print(response)

Thought: The user wants to know what Lyft did in R&D in 2022 versus 2021, which requires accessing information from the annual reports.
Action: lyft_2022_10k_form
Action Input: {'input': 'R&D expenses'}
Observation: $856,777
Thought: The user wants to know what Lyft did in R&D in 2022 versus 2021, and I've already accessed the information from the annual report for 2022. Next, I need to access the similar information from the annual report for 2021.
Action: lyft_2021_10k_form
Action Input: {'input': 'R&D expenses'}
Observation: Research and development expenses primarily consist of personnel-related compensation costs and facilities costs. Such expenses include costs related to autonomous vehicle technology initiatives. Research and development costs are expensed as incurred.
Thought: The user wants to know what Lyft did in R&D in 2022 versus 2021, and I've accessed the information from both annual reports. Now, I need to compare the expenses.
Action: lyft_2020_10k_form
Action Input: {